# Mathematical model kinetics

In [46]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions
#from phd.visualization.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq
import vessel_manoeuvring_models
import re
import os

In [48]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")
#model_loaders = "semiempirical rudder" : catalog.load(f"{ship}.models_VCT_MDL")['semiempirical_covered_inertia'],
#models_rudder_VCT_loaders = catalog.load(f"{ship}.models_rudder_VCT")

[12/03/24 09:45:20] INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=901549;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=840113;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [49]:
model = catalog.load(f"{ship}.models_VCT")['semiempirical_covered_inertia']()

                    INFO     Loading data from 'wPCC.models_VCT' (PartitionedDataset)...        ]8;id=917563;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=767031;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [50]:
eq_X_D = remove_functions(model.X_D_eq)

In [51]:
eq_X = sp.Eq(model.X_eq.lhs,model.X_eq.rhs.subs(model.X_D_eq.rhs,X_D_))
eq_Y = sp.Eq(model.Y_eq.lhs,model.Y_eq.rhs.subs(model.Y_D_eq.rhs,Y_D_))
eq_N = sp.Eq(model.N_eq.lhs,model.N_eq.rhs.subs(model.N_D_eq.rhs,N_D_))

#save_eq(remove_functions(eq_X), file_name='X')
#save_eq(remove_functions(eq_Y), file_name='Y')
#save_eq(remove_functions(eq_N), file_name='N')

In [52]:
X_D_eq = remove_functions(model.X_D_eq).subs([
    ("X_W",0),
    ("X_RHI",0),
])
Y_D_eq = remove_functions(model.Y_D_eq).subs("Y_W",0)
N_D_eq = remove_functions(model.N_D_eq).subs("N_W",0)

save_eq(remove_functions(X_D_eq), file_name='X_D')
save_eq(remove_functions(Y_D_eq), file_name='Y_D')
save_eq(remove_functions(N_D_eq), file_name='N_D')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

'N_{D} = N_{H} + N_{P} + N_{R} + N_{RHI}'

In [53]:
model.show_subsystems()

hull: PrimeEquationSubSystem
propellers: PropellersSimpleSystem
rudder_port: SemiempiricalRudderSystemCovered
rudder_stbd: SemiempiricalRudderSystemCovered
rudders: Rudders
rudder_hull_interaction: RudderHullInteractionSystem
wind_force: DummyWindForceSystem


## Hull

In [54]:
from phd.paper.equation import to_latex
from IPython.display import display, Latex, Math, HTML

eq_latex = to_latex(model.subsystems['hull'].equations['Y_H'])

eq_latex = eq_latex.replace(r'Y_{rrr} r^{3}',r'\cancel{Y_{rrr}} r^{3}')

In [55]:
def cancel(eq:sp.Eq,cancels = []):
    subs={}
    for cancel in cancels:
        subs[cancel]=sp.Symbol(fr'\cancel{{{cancel}}}')
    eq = eq.subs(subs)
    return eq

In [56]:
cancels = [
    #p.Yrrr,p.Yvrr,p.Yvvr,p.Yvvv,
    #p.Nrrr,p.Nvrr,p.Nvvr,p.Nvvv,
]
for name, eq in model.subsystems['hull'].equations.items():
    #save_eq(cancel(prime(eq)), file_name=name)
    save_eq(prime(cancel(eq, cancels=cancels)), file_name=name)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

## Propellers

In [57]:
for name, eq in model.subsystems['propellers'].equations.items():
        
    if isinstance(model.subsystems['propellers'], vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem):
        save_eq(prime(eq), file_name=name)
    else:
        save_eq(eq, file_name=name)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

## Rudder hull interaction

In [58]:
for name, eq in model.subsystems['rudder_hull_interaction'].equations.items():
    if isinstance(model.subsystems['rudder_hull_interaction'], vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem):
        save_eq(prime(eq), file_name=name)
    else:
        save_eq(eq, file_name=name)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

## Semi-empirical rudder model

In [59]:
rudder = model.subsystems['rudder_port']
#rudder.equations['r_p_port'] = rudder.equations['r_p_port'].subs('r_p_port','r_x_port')

In [60]:
rudder.equations['X_R_port']

Eq(X_R_port, -D_R_port*cos(alpha_f_port) + L_R_port*sin(alpha_f_port))

In [61]:
keys = list(rudder.equations.keys())
keys.reverse()

### Remove stall

In [62]:
rudder.equations['C_L_port'] = sp.Eq(rudder.equations['C_L_port'].lhs,rudder.equations['C_L_port'].rhs.args[0][0])

In [63]:
rudder.equations['C_L_port']

Eq(C_L_port, K_gap_port*(alpha_port*dC_L_dalpha_port + C_D_crossflow_port*alpha_port*Abs(alpha_port)/AR_e_port))

In [64]:
C_L_tune = sp.symbols("C_L_tune")

In [65]:
rudder.equations['L_R_C_port'] = rudder.equations['L_R_C_port'].subs(C_L_tune,1)
rudder.equations['L_R_U_port'] = rudder.equations['L_R_U_port'].subs(C_L_tune,1)

In [66]:
C_D_crossflow_port = symbols("C_D_crossflow_port")  # crossflow drag coefficient
delta = symbols("delta")  # crossflow drag coefficient

lambda_gap_port,K_gap = symbols(
    "lambda_gap_port, K_gap"
)

subs={
    C_D_crossflow_port : symbols("C_DC"),
    lambda_gap_port: K_gap,
    
}
for name,eq in rudder.equations.items():
    rudder.equations[name] = eq.subs(subs)


In [67]:
rudder.equations["C_L_port"]

Eq(C_L_port, K_gap_port*(alpha_port*dC_L_dalpha_port + C_DC*alpha_port*Abs(alpha_port)/AR_e_port))

In [68]:
replace_latex = {
    r"dC_{L dalpha }" : r"\frac{\partial C_L}{\partial \alpha}",
}
for name in keys:
    eq = rudder.equations[name]
    name_clean = name.replace('_port','')
    file_name = f"{name_clean}_semiempirical" 
    save_eq(eq, file_name=file_name, subs={'port':''}, replace_latex=replace_latex)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

In [69]:
save_eq(eq, file_name=file_name, subs={'port':''}, replace_latex=replace_latex)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

'V_{A } = u \\left(1 - w_{f}\\right)'

In [70]:
s_latex = ""
for name in keys:
    eq = rudder.equations[name]
    name_clean = name.replace('_port','')
    file_name = f"{name_clean}_semiempirical"
    
    eq_latex = fr"""
\begin{{equation}}
\label{{eq:{file_name}}}
\input{{equations/mathematical_model_kinetics.{file_name}}}
\end{{equation}}
%"""
    s_latex+=eq_latex

print(s_latex)


\begin{equation}
\label{eq:N_R_semiempirical}
\input{equations/mathematical_model_kinetics.N_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:Y_R_semiempirical}
\input{equations/mathematical_model_kinetics.Y_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:X_R_semiempirical}
\input{equations/mathematical_model_kinetics.X_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:D_R_semiempirical}
\input{equations/mathematical_model_kinetics.D_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:C_D_U_semiempirical}
\input{equations/mathematical_model_kinetics.C_D_U_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:C_D0_U_semiempirical}
\input{equations/mathematical_model_kinetics.C_D0_U_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:C_F_U_semiempirical}
\input{equations/mathematical_model_kinetics.C_F_U_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:Re_F_U_semiempirical}
\input{equations/mathematical_model_kinetics.Re

## Nomenclature

In [71]:
df_nomenclature = pd.DataFrame(columns=['LaTeX','Description','file_names'])
for lhs, eq in paper.equations.items():
    for symbol in eq['equation'].free_symbols:
        if symbol in df_nomenclature.index:
            df_nomenclature.loc[symbol,'file_names'].append(eq['file_name'])
        else:
            df_nomenclature.loc[symbol,'file_names'] = [eq['file_name']]

subs = {sp.Symbol(str(row['symbol'])):sp.Symbol(str(row['symbol']).replace('delta',r'\delta')) for name, row in df_parameters.iterrows()}
parameters = [prime(symbol.subs(subs)) for name,symbol in p.items()]

index = list(set(df_nomenclature.index) - set(parameters))
df_nomenclature = df_nomenclature.loc[index].copy()

df_nomenclature.index = [sp.Symbol(symbol.name.replace('_port','')) for symbol in df_nomenclature.index]
df_nomenclature.index = [sp.Symbol(symbol.name.replace('_stbd','')) for symbol in df_nomenclature.index]
df_nomenclature.index = [sp.Symbol(symbol.name.replace('port','')) for symbol in df_nomenclature.index]
df_nomenclature.index = [sp.Symbol(symbol.name.replace('stbd','')) for symbol in df_nomenclature.index]
df_nomenclature.index = [sp.Symbol(symbol.name.replace("'",'')) for symbol in df_nomenclature.index] 

df_nomenclature['LaTeX'] = [f"${paper.to_latex(symbol)}$" for symbol in df_nomenclature.index]
#df_nomenclature.sort_values(by='LaTeX', key=lambda col: col.str.lower(), inplace=True)
#
#df_nomenclature.index = df_nomenclature.index.astype(str)
#df_nomenclature.index = [re.sub(r"^{", "", s) for s in df_nomenclature.index]
#df_nomenclature.index = [re.sub(r"}$", "", s) for s in df_nomenclature.index]
#df_nomenclature.index = [s.replace("\\","") for s in df_nomenclature.index]
#
#mask = df_nomenclature['LaTeX'].str.contains(r'\\cancel')
#df_nomenclature = df_nomenclature.loc[~mask].copy()

In [72]:
df_nomenclature_new = pd.DataFrame(columns=df_nomenclature.columns)

files = {}

for symbol, row in df_nomenclature.iterrows():

    if symbol in df_nomenclature_new.index:
        continue

    df_nomenclature_new.loc[symbol] = row

    
    if len(df_nomenclature.loc[symbol]) > 1:
        file_names_all = []
        for file_names in df_nomenclature.loc[symbol,'file_names']:
            if isinstance(file_names, list):
                file_names_all+=file_names
            else:
                file_names_all.append(file_names)

        files[symbol] = list(set(file_names_all))

df_nomenclature_new['file_names'] = files

### Parse the LaTeX for equation references

In [73]:
s = r"\input{equations/mathematical_model_kinetics.gamma_g_semiempirical}"
result = re.search(r"\\input\{equations/mathematical_model_kinetics\.([^}]+)\}", s)
result.group(1)

'gamma_g_semiempirical'

In [74]:
for result in re.finditer(r"\\input\{equations/mathematical_model_kinetics\.([^}]+)\}", s):
    print(result.group(1))

gamma_g_semiempirical


In [75]:
paper.paper_path

'/home/maa/dev/PHD/docs/paper5'

In [76]:
equations = []
for file_name in os.listdir(paper.paper_path):
    if not os.path.splitext(file_name)[-1] == '.tex':
        continue

    file_path = os.path.join(paper.paper_path, file_name)
    with open(file_path, mode='r') as file:
        s = file.read()

    for result in re.finditer(r"\\input\{equations/mathematical_model_kinetics\.([^}]+)\}", s):
        equations.append(result.group(1))

equations = set(equations)

In [78]:
mask_active_symbols = [len(set(row['file_names']) & equations) > 0 for index, row in df_nomenclature_new.iterrows()]
df_nomenclature_new = df_nomenclature_new.loc[mask_active_symbols].copy()
if 'dC_L_dalpha' in df_nomenclature_new.index:
    df_nomenclature_new.drop(index=['dC_L_dalpha'], inplace=True)

In [79]:
descriptions = {
    'alpha' : 'Rudder angle of attack',
    'alpha_f': 'Rudder inflow angle',
    'delta': 'Rudder angle',
    'delta_lim': 'Rudder gap limit',
    'dot{r': 'Yaw acceleration',
    'dot{u': 'Surge acceleration',
    'dot{v': 'Sway acceleration',
    'gamma': 'Prop. induced angle',
    'gamma_0': 'Rudder initial inflow',
    'gamma_g': 'Geometric inflow angle',
    'kappa_r_gamma_g': '$\kappa_r$ nonlinear',
    'kappa_r_tot': '$\kappa_r$ total',
    'kappa_r': 'Yaw rate flow straightening',
    'kappa_v_gamma_g': '$\kappa_v$ nonlinear',
    'kappa_v_tot': '$\kappa_v$ total',
    'kappa_v': 'Drift flow straightening',
    'lambda_gap': 'Rudder gap effec',
    'lambda_R': 'limited radius of the prop. slipstream',
    'lambda_': 'Rudder taper ratio',
    'nu': 'Kinematic viscosity',
    'Omega': 'Rudder sweep angle',
    'rho': 'Water density',
    'a_0': 'Section lift curve slope',
    'a_H': 'Sway rudder interaction',
    'A_R_C': 'Rudder area covered',
    'A_R_U': 'Rudder area uncovered',
    'A_R': 'Rudder area',
    'AR_e': 'Effective aspect ratio',
    'AR_g': 'Geometric aspect ratio',
    'b_R': 'Rudder span',
    'c': 'Rudder chord',
    'C_D_C': 'Drag coeff. covered',
    'C_D_tune': 'Drag coeff. tunnig',
    'C_D_U': 'Drag coeff. uncovered',
    'C_D0_C': 'Initial drag coeff. covered',
    'C_D0_tune': 'Initial drag coeff. tune',
    'C_D0_U': 'Initial drag coeff. uncovered',
    'C_DC': 'Rudder cross-flow drag',
    'C_F_C': 'Friction coeff. covered',
    'C_F_U': 'Friction coeff. uncovered',
    'C_L': 'Lift coeff.',
    'c_r': 'Rudder root choord',
    'C_Th': 'Thrust coeff.',
    'c_t': 'Rudder tip choord',
    'd': 'lift diminished variable',
    'D_R': 'Rudder drag',
    'dC_L_dalpha': 'Lift slope',
    'e_0': 'Oswald efficiency factor',
    'f': 'lift diminished variable',
    'I_z': 'Yaw mass intertia',
    'L_R_C': 'Lift covered part',
    'L_R_U': 'Lift uncovered part',
    'L_R': 'Rudder lift',
    'm': 'Ship mass',
    'N_{dot{r}': 'Yaw added mass',
    'N_{dot{v}': 'Yaw added mass',
    'N_D': 'Yaw damping moment',
    'N_H': 'Hull yawing moment',
    'N_P': 'Prop. yawing moment',
    'N_RHI': 'Rudder hull interaction',
    'N_R': 'Rudder yawing moment',
    'r': 'Yaw rate',
    'r_0': 'Prop. radius',
    'r_Delta': 'Turbulent mixing',
    'r_infty': 'Prop. slipstream',
    'r_x': 'Prop. slipstream at rudder',
    'Re_F_C': 'Reynold number covered',
    'Re_F_U': 'Reynold number uncovered',
    's': 'Rudder gap coefficient',
    'thrust': 'Thrust',
    'u': 'Surge vel.',
    'v': 'Sway vel.',
    'V_infty': 'Prop. slipstream',
    'V_A': 'Apparent vel.',
    'V_R_C': 'Rudder vel. covered',
    'V_R_U': 'Rudder vel. uncovered',
    'V_R_x_C': 'Axial vel. covered',
    'V_R_x_U': 'Axial vel. uncovered',
    'V_R_y': 'Rudder transverse vel.',
    'V_x_corr': 'Turbulent mixing',
    'V_x_C': 'No turbulent mixing',
    'V_x_U': 'No turbulent mixing',
    'w_f': 'Taylor wake',
    'x': 'Prop. to rudder',
    'X_{dot{u}': 'Surge added mass',
    'X_D': 'Surge damping force',
    'x_G': 'Center of gravity',
    'x_H': 'Yaw rudder interaction',
    'X_H': 'Hull surge force',
    'X_P': 'Prop. surge force',
    'x_R': 'Rudder lever',
    'X_R': 'Rudder surge force',
    'X_{thrust': 'Thrust deduction',
    'Y_{dot{r}': 'Sway added mass',
    'Y_{dot{v}': 'Sway added mass',
    'Y_D': 'Sway damping force',
    'Y_H': 'Hull sway force',
    'y_p': 'Propeller lever',
    'Y_P': 'Propeller sway force',
    'Y_RHI': 'Rudder hull interaction',
    'Y_R': 'Rudder sway force',
    'y_R': 'Rudder lever'
}
descriptions=pd.Series(descriptions)

In [80]:
#{f"{index}":'' for index in df_nomenclature_new.index}

In [81]:
df_nomenclature_new.index

Index([], dtype='object')

In [82]:
df_nomenclature_new['Description'] = descriptions

In [83]:
df_nomenclature_new

,LaTeX,Description,file_names
alpha,NaN,Rudder angle of attack,NaN
alpha_f,NaN,Rudder inflow angle,NaN
delta,NaN,Rudder angle,NaN
delta_lim,NaN,Rudder gap limit,NaN
dot{r,NaN,Yaw acceleration,NaN
dot{u,NaN,Surge acceleration,NaN
dot{v,NaN,Sway acceleration,NaN
gamma,NaN,Prop. induced angle,NaN
gamma_0,NaN,Rudder initial inflow,NaN
gamma_g,NaN,Geometric inflow angle,NaN


In [84]:
table = paper.table_columns(df_nomenclature_new[['LaTeX','Description']], columns=3)
table.to_csv(paper.file_path_with_nb_ref("nomenclature.csv", directory='tables'), index=False)

In [85]:
table

,LaTeX,Description,LaTeX1,Description1,LaTeX2,Description2
0,NaN,Rudder angle of attack,NaN,Initial drag coeff. covered,NaN,Rudder gap coefficient
1,NaN,Rudder inflow angle,NaN,Initial drag coeff. tune,NaN,Thrust
2,NaN,Rudder angle,NaN,Initial drag coeff. uncovered,NaN,Surge vel.
3,NaN,Rudder gap limit,NaN,Rudder cross-flow drag,NaN,Sway vel.
4,NaN,Yaw acceleration,NaN,Friction coeff. covered,NaN,Prop. slipstream
5,NaN,Surge acceleration,NaN,Friction coeff. uncovered,NaN,Apparent vel.
6,NaN,Sway acceleration,NaN,Lift coeff.,NaN,Rudder vel. covered
7,NaN,Prop. induced angle,NaN,Rudder root choord,NaN,Rudder vel. uncovered
8,NaN,Rudder initial inflow,NaN,Thrust coeff.,NaN,Axial vel. covered
9,NaN,Geometric inflow angle,NaN,Rudder tip choord,NaN,Axial vel. uncovered
